# Preparation

In [ ]:
import subprocess
import threading

#istallazione di ollama
!curl -fsSL https://ollama.com/install.sh | sh# This Python 3 environment comes with many helpful analytics libraries installed

In [ ]:
def start_ollama():
    t = threading.Thread(target=lambda: subprocess.run(["ollama", "serve"]),daemon=True)
    t.start()

In [ ]:
def pull_model(local_llm):
    !ollama pull local_llm

In [ ]:
def start_model(local_llm):        
    t2 = threading.Thread(target=lambda: subprocess.run(["ollama", "run", local_llm]),daemon=True)
    t2.start()

In [ ]:
%%capture --no-stderr
%pip install -U scikit-learn==1.3 langchain-ai21 langchain_community tiktoken langchainhub langchain langgraph

In [ ]:
import os

os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tqdm import tqdm

# Dati

In [3]:
# todo

x_test = None
y_test = None

# Hate speech

In [2]:
model = "llama3.1"

start_ollama()
pull_model(model_name)
start_model(model_name)

NameError: name 'start_ollama' is not defined

In [ ]:
prompt = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an helpful assistant who has to detect the presence of hate speech.
            Hate speech is speech that attacks a person or group based on attributes such as race, religion, ethnic origin, national origin, sex, disability, sexual orientation, or gender identity. 
            You have to answer "yes" if it contains hate speech, or "no" if it doesn't contain hate speech. NO PREAMBLE, NO EXPLANATIONS.
            <|eot_id|><|start_header_id|>user<|end_header_id|> 
            Do you think this document contain hate speech? document: {document}.
            <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["document"],
    )

In [ ]:
def hate_speech_detection(prompt, model):
    llm = ChatOllama(model=model, temperature=0)
    hate_speech_detection = prompt | llm | StrOutputParser()
    return hate_speech_detection

In [ ]:
# Label 0: hate speech (yes answer)
# Label 1: no-hate speech (no answer)

def predict(prompt,model,x_test):
    y_pred = []
    for x in tqdm(x_test):
        answer = hate_speech_detection(prompt,model).invoke({"document":x})
        if answer.lower() == "yes": y_pred.append(0)
        elif answer.lower() == "no": y_pred.append(1)
    return y_pred

y_pred = predict(prompt,model,x_test)

In [ ]:
# Valutazione del modello
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{class_report}")